### Stockage de données

##### 1. Chargement de données

In [8]:
import pandas as pd

data_train = pd.read_csv('../data/processed/preprocessed_train_text.csv')
data_test = pd.read_csv('../data/processed/preprocessed_test_text.csv')


##### 2. Extraire et sauvegarder les métadonnées nécessaires (label, identifiant).

In [10]:
train_metadata = data_train[["label"]].copy()
test_metadata = data_test[["label"]].copy()

train_metadata["id"] = range(len(train_metadata))
test_metadata["id"] = range(len(test_metadata))

- Sauvegarder

In [11]:
import os

os.makedirs('../data/metadata')

train_metadata.to_csv("../data/metadata/train_metadata.csv", index=False)
test_metadata.to_csv("../data/metadata/test_metadata.csv", index=False)

##### 2. Créer les collections ChromaDB

- Initialisation

In [12]:
import chromadb

client = chromadb.Client()

- Pour les données d’entraînement.

In [13]:
train_collection = client.create_collection(
    name="train_collection",
    metadata={"description": "Embeddings pour données d'entraînement"}
)

- Pour les données de test.

In [14]:
test_collection = client.create_collection(
    name="test_collection",
    metadata={"description": "Embeddings pour données de test"}
)

##### 3. Insérer les embeddings et métadonnées dans les collections.

- la colonne `ids` doit toujours être une liste de chaînes de caractères (str), pas des entiers.
- `ids` doit être unique pour chaque document.
- `embeddings` doit être une liste de listes, pas un array Numpy.
- Diviser les données en plus petits lots avant de les ajouter à la collection. (ValueError si les lignes se dépasse 5461)

1. Charger les embeddings

In [18]:
import numpy as np

train_embeddings = np.load('../data/embeddings/train_embeddings_minilm.npy')
test_embeddings = np.load('../data/embeddings/test_embeddings_minilm.npy')


2. Train

In [35]:
batch_size = 5000
n = len(train_embeddings)

for i in range(0, n, batch_size):
    end = min(i + batch_size, n)
    
    batch_ids = [str(j) for j in list(train_metadata['id'][i: end])]
    batch_embeddings = train_embeddings[i:end].tolist()
    batch_metadatas = [{"label": int(train_metadata['label'][j])} for j in range(i, end)]
    
    train_collection.add(
        ids=batch_ids,
        embeddings=batch_embeddings,
        metadatas=batch_metadatas
    )
    
    print(f"Inserted {end} / {n}")

Inserted 5000 / 120000
Inserted 10000 / 120000
Inserted 15000 / 120000
Inserted 20000 / 120000
Inserted 25000 / 120000
Inserted 30000 / 120000
Inserted 35000 / 120000
Inserted 40000 / 120000
Inserted 45000 / 120000
Inserted 50000 / 120000
Inserted 55000 / 120000
Inserted 60000 / 120000
Inserted 65000 / 120000
Inserted 70000 / 120000
Inserted 75000 / 120000
Inserted 80000 / 120000
Inserted 85000 / 120000
Inserted 90000 / 120000
Inserted 95000 / 120000
Inserted 100000 / 120000
Inserted 105000 / 120000
Inserted 110000 / 120000
Inserted 115000 / 120000
Inserted 120000 / 120000


3. Test

In [36]:
batch_size = 5000
n = len(test_embeddings)

for i in range(0, n, batch_size):
    end = min(i + batch_size, n)
    
    batch_ids = [str(j) for j in list(test_metadata['id'][i: end])]
    batch_embeddings = test_embeddings[i:end].tolist()
    batch_metadatas = [{"label": int(test_metadata['label'][j])} for j in range(i, end)]
    
    test_collection.add(
        ids=batch_ids,
        embeddings=batch_embeddings,
        metadatas=batch_metadatas
    )
    
    print(f"Inserted {end} / {n}")

Inserted 5000 / 7600
Inserted 7600 / 7600
